In [1]:
from bggcomplex import BGGComplex
from fast_module import FastLieAlgebraCompositeModule, FastModuleFactory, BGGCohomology
import numpy as np
%load_ext cython

In [2]:
for d in ['A1','A2','A3','B2','G2','A4','B3','C3']:
    BGG = BGGComplex(d)
    f_factory = FastModuleFactory(BGG.LA)
    component_dic = {'b':f_factory.build_component('b','ad'),'u':f_factory.build_component('u','coad')
                     ,'b+':f_factory.build_component('b+','coad'),'n':f_factory.build_component('n','ad')}
    components = [[("n",1,'wedge'),("n",1,'wedge')]]

    module = FastLieAlgebraCompositeModule(f_factory, components, component_dic)

    cohom = BGGCohomology(BGG, module)
    for i in range(10):
        cohom.cohomology_LaTeX(i,only_non_zero=True,complex_string=d+r'/B, \mathfrak n\otimes \mathfrak n')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [30]:
len(component_dic['b'].basis)

14

In [10]:
d='A2'
BGG = BGGComplex(d)
f_factory = FastModuleFactory(BGG.LA)
component_dic = {'b':f_factory.build_component('b','ad'),'u':f_factory.build_component('u','coad')
                 ,'b+':f_factory.build_component('b+','coad'),'n':f_factory.build_component('n','ad')}
components = [[("u",5,'wedge')]]

module = FastLieAlgebraCompositeModule(f_factory, components, component_dic)

cohom = BGGCohomology(BGG, module)
for i in range(10):
    cohom.cohomology_LaTeX(i,only_non_zero=True,complex_string=d+r'/B, \wedge^2\mathfrak u')

ValueError: Cannot take 5-fold wedge power of 3-dimensional module u